In [ ]:
!pip install SpeechRecognition pyttsx3
!apt-get install -y mpg321
!pip install googletrans==4.0.0-rc1
!pip install requests
!pip install gtts
!pip install IPython
!apt-get install espeak
!pip install pydub
!apt-get install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 16.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libao-common libao4 libaudio-scrobbler-perl libauthen-sasl-perl libclone-perl
  libconfig-inifiles-perl libdata-dump-perl libencode-locale-perl libfile-listing-perl
  libfont-afm-perl libhtml-form-perl libhtml-format-perl libhtml-parser-perl libhtml-tagset-perl
  libhtml-tree-perl libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl libhttp-message-perl
  libhttp-negotiate-perl libid3tag0 libio-html-perl libio-socket-ssl-perl liblwp-mediatypes-perl
  liblwp-protocol-https-perl libmad0 libmailtools-perl libnet-http-perl libnet-smtp-ssl-perl
  libnet-ssleay-perl libtry-tiny-perl liburi-perl libwww-perl libwww-robotrules-perl netbase
  perl-openssl-defaults
Suggested packages:
  libaudio2 libsndio6.1 libdigest-hmac-perl libgssapi-perl libcrypt-ssleay-perl 

In [ ]:
from google.colab import files
import speech_recognition as sr
import requests
from gtts import gTTS
from googletrans import Translator
from pydub import AudioSegment
from IPython.display import Audio, display

# API keys
NewsAPI = "ADD_YOUR_KEY_HERE"
AviationStack = "ADD_YOUR_KEY_HERE"

# Function to convert m4a to wav using pydub
def convert_m4a_to_wav(m4a_file_path):
    audio = AudioSegment.from_file(m4a_file_path, format="m4a")
    wav_file_path = m4a_file_path.replace(".m4a", ".wav")
    audio.export(wav_file_path, format="wav")
    return wav_file_path

# Function to convert speech to text
def recognize_speech_from_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        print(f"Recognized Speech: {text}")
        return text.lower()
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from the speech recognition service; {e}")
        return None

# Function to process commands from speech
def virtual_assistant(audio_file):
    command = recognize_speech_from_audio(audio_file)

    if command:
        if "translate" in command and "arabic" in command:
            text = input("Enter the text to translate: ")
            translation = translate_to_arabic(text)
            print(f"Translated text: {translation}")

        elif "news" in command or "headlines" in command:
            headlines = get_headlines()
            if headlines:
                print("Here are the latest headlines:")
                for i, headline in enumerate(headlines, 1):
                    print(f"{i}. {headline}")

                # Only read headlines aloud if specifically requested
                if "read" in command:
                    read_headlines(headlines)
            else:
                print("Failed to retrieve headlines.")

        elif "flight" in command and "details" in command:
            flight_number = input("Enter the flight number (IATA): ")
            flight_details = get_flight_details(flight_number)
            if flight_details:
                print("Flight details:")
                for key, value in flight_details.items():
                    print(f"{key.replace('_', ' ').capitalize()}: {value}")
            else:
                print("Failed to retrieve flight details.")
        else:
            print("Command not recognized.")
    else:
        print("No command recognized from the audio.")

# Function to translate English text to Arabic
def translate_to_arabic(text):
    translator = Translator()
    translation = translator.translate(text, dest='ar')
    return translation.text

# Function to get the latest news headlines
def get_headlines():
    url = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={NewsAPI}'
    response = requests.get(url)
    headlines = []
    if response.status_code == 200:
        data = response.json()
        for article in data['articles']:
            headlines.append(article['title'])
    return headlines

# Function to get flight details using AviationStack API
def get_flight_details(flight_number):
    url = f'http://api.aviationstack.com/v1/flights?access_key={AviationStack}&flight_iata={flight_number}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'data' in data and len(data['data']) > 0:
            flight_info = data['data'][0]
            details = {
                'airline': flight_info['airline']['name'],
                'departure_airport': flight_info['departure']['airport'],
                'arrival_airport': flight_info['arrival']['airport'],
                'departure_time': flight_info['departure']['scheduled'],
                'arrival_time': flight_info['arrival']['scheduled'],
                'flight_status': flight_info['flight_status']
            }
            return details
        else:
            print("No flight data found for the given flight number.")
            return None
    else:
        print(f"Failed to retrieve flight details. Status code: {response.status_code}")
        print(f"Response content: {response.text}")
        return None

# Function to read headlines aloud using gTTS and display in audio widget
def read_headlines(headlines):
    all_headlines = ' '.join(headlines)
    tts = gTTS(text=all_headlines, lang='en')
    tts.save("headlines.mp3")
    print("Reading the latest headlines aloud:")
    display(Audio("headlines.mp3", autoplay=True))

# Start of main program loop
print("Hello! How can I help you today?")
while True:
    uploaded = files.upload()

    # Extract the file name of the uploaded file
    audio_file_path = list(uploaded.keys())[0]

    # Convert m4a to wav if necessary
    if audio_file_path.endswith(".m4a"):
        audio_file_path = convert_m4a_to_wav(audio_file_path)

    # Run the virtual assistant using the converted audio file
    virtual_assistant(audio_file_path)

    # Prompt to continue or stop, with re-prompt if unrecognized input
    while True:
        cont = input("\n\nDo you want to continue? Type 'yes' to continue or 'stop' to stop the program: ").strip().lower()
        if cont == "stop":
            print("Goodbye!")
            break  # Exit the inner while loop
        elif cont == "yes":
            break  # Continue to the next iteration of the outer loop
        else:
            print("I do not understand. Please try again.")

    # If "stop" was entered, break out of the main loop
    if cont == "stop":
        break


Hello! How can I help you today?


Saving Get the latest news headlines.m4a to Get the latest news headlines.m4a
Recognized Speech: get the latest news headlines
Here are the latest headlines:
1. [Removed]
2. Stock market today: Dow, S&P 500, Nasdaq soar as Trump triumphs - Yahoo Finance
3. Ukraine says forces clash with North Korean troops for first time - The Washington Post
4. Tesla Shares Hit Record—Soaring 12%—And Elon Musk Becomes $15 Billion Richer With Trump Victory - Forbes
5. 2024 Week 10 NFL score predictions, game picks, odds: Model releases exact scores from 10,000 simulations - CBS Sports
6. Detroit Lions mailbag: How Za’Darius Smith will impact depth chart, scheme - Pride Of Detroit
7. [Removed]
8. NFL trade deadline winners and losers: While Chiefs and Lions bolster Super Bowl hopes, Cowboys sacrifice only their future - Yahoo Sports
9. Marijuana legalization fails in Florida as numerous states approve citizen voting amendments - PBS NewsHour
10. AMD Ryzen 7 9800X3D Linux Performance: Zen 5 With 3D V-Cac

Saving Read the headlines.m4a to Read the headlines.m4a
Recognized Speech: read the headlines
Here are the latest headlines:
1. [Removed]
2. Stock market today: Dow, S&P 500, Nasdaq soar as Trump triumphs - Yahoo Finance
3. Ukraine says forces clash with North Korean troops for first time - The Washington Post
4. Tesla Shares Hit Record—Soaring 12%—And Elon Musk Becomes $15 Billion Richer With Trump Victory - Forbes
5. 2024 Week 10 NFL score predictions, game picks, odds: Model releases exact scores from 10,000 simulations - CBS Sports
6. Detroit Lions mailbag: How Za’Darius Smith will impact depth chart, scheme - Pride Of Detroit
7. [Removed]
8. NFL trade deadline winners and losers: While Chiefs and Lions bolster Super Bowl hopes, Cowboys sacrifice only their future - Yahoo Sports
9. Marijuana legalization fails in Florida as numerous states approve citizen voting amendments - PBS NewsHour
10. AMD Ryzen 7 9800X3D Linux Performance: Zen 5 With 3D V-Cache Review - Phoronix
11. NASA's P



Do you want to continue? Type 'yes' to continue or 'stop' to stop the program: yes


Saving Show me the flight details.m4a to Show me the flight details.m4a
Recognized Speech: show me the flight details
Enter the flight number (IATA): AA100
Flight details:
Airline: American Airlines
Departure airport: John F Kennedy International
Arrival airport: Heathrow
Departure time: 2024-11-06T19:00:00+00:00
Arrival time: 2024-11-07T06:50:00+00:00
Flight status: landed


Do you want to continue? Type 'yes' to continue or 'stop' to stop the program: yes


Saving Translate this to Arabic.m4a to Translate this to Arabic.m4a
Recognized Speech: translate this to Arabic
Enter the text to translate: Hello good morning
Translated text: مرحبا صباح الخير


Do you want to continue? Type 'yes' to continue or 'stop' to stop the program: stop
Goodbye!
